<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### Capital Market Theory

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## CMT Over Time

Topics of interest include:

* CAPM
 * in-sample approach
 * out-of-sample testing
* APT
 * in-sample approach
 * out-of-sample testing

## Real Financial Data

**_Historical end-of-day financial time series data._**

See Artificial Intelligence in Finance (ch. 04)  and `http://hilpisch.com/aiif_eikon_eod_data.csv`.

## Imports and Data

In [ ]:
!git clone https://github.com/tpq-classes/python_for_asset_management.git
import sys
sys.path.append('python_for_asset_management')


In [ ]:
import math
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')

In [ ]:
raw = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_data.csv',
                  index_col=0, parse_dates=True).dropna()

In [ ]:
rets = np.log(raw / raw.shift(1)).dropna()

In [ ]:
rets.info()

## Capital Asset Pricing Model

In [ ]:
r = 0.005

In [ ]:
market = '.SPX'

In [ ]:
res = pd.DataFrame()

In [ ]:
for sym in rets.columns[:4]:
    print('\n' + sym)
    print(54 * '=')
    for year in range(2010, 2019):
        rets_ = rets.loc[f'{year}-01-01':f'{year}-12-31']
        muM = rets_[market].mean() * 252
        cov = rets_.cov().loc[sym, market]
        var = rets_[market].var()
        beta = cov / var
        rets_ = rets.loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
        muM = rets_[market].mean() * 252
        mu_capm = r + beta * (muM - r)
        mu_real = rets_[sym].mean() * 252
        res = pd.concat([res, pd.DataFrame({'symbol': sym,
                                       'mu_capm': mu_capm,
                                       'mu_real': mu_real},
                                      index=[year + 1])],
                        sort=True)
        print('{} | beta: {:.3f} | mu_capm: {:6.3f} | mu_real: {:6.3f}'
              .format(year + 1, beta, mu_capm, mu_real))

In [ ]:
sym = 'AMZN.O'
# sym = 'INTC.O'

In [ ]:
res[res['symbol'] == sym][['mu_capm', 'mu_real']].corr()

In [ ]:
res[res['symbol'] == sym].plot(kind='bar',
                figsize=(10, 6), title=sym);

In [ ]:
grouped = res.groupby('symbol').mean()
grouped

In [ ]:
grouped.plot(kind='bar', figsize=(10, 6), title='Average Values');

## Arbitrage-Pricing Theory

### Simple Factors

In [ ]:
factors = ['.SPX', '.VIX', 'EUR=', 'XAU=']

In [ ]:
res = pd.DataFrame()

In [ ]:
np.set_printoptions(formatter={'float': lambda x: f'{x:5.2f}'})

In [ ]:
for sym in rets.columns[:4]:
    print('\n' + sym)
    print(71 * '=')
    for year in range(2010, 2019):
        rets_ = rets.loc[f'{year}-01-01':f'{year}-12-31']
        reg = np.linalg.lstsq(rets_[factors],
                              rets_[sym], rcond=-1)[0]
        rets_ = rets.loc[f'{year + 1}-01-01':f'{year + 1}-12-31']
        mu_apt = np.dot(rets_[factors].mean() * 252, reg)
        mu_real =  rets_[sym].mean() * 252
        res = pd.concat([res, pd.DataFrame({'symbol': sym,
                        'mu_apt': mu_apt, 'mu_real': mu_real},
                         index=[year + 1])])
        print('{} | fl: {} | mu_apt: {:6.3f} | mu_real: {:6.3f}'
              .format(year + 1, reg.round(2), mu_apt, mu_real))

In [ ]:
sym = 'AMZN.O'
sym = 'INTC.O'

In [ ]:
res[res['symbol'] == sym][['mu_apt', 'mu_real']].corr()

In [ ]:
res[res['symbol'] == sym].plot(kind='bar',
                figsize=(10, 6), title=sym);

In [ ]:
grouped = res.groupby('symbol').mean()
grouped

In [ ]:
grouped.plot(kind='bar', figsize=(10, 6), title='Average Values');

### Typical Factors

In [ ]:
factors = pd.read_csv('http://hilpisch.com/aiif_eikon_eod_factors.csv',
                      index_col=0, parse_dates=True)

In [ ]:
factors.info()

In [ ]:
(factors / factors.iloc[0]).plot(figsize=(10, 6));

In [ ]:
start = '2017-01-01'
end = '2020-01-01'

In [ ]:
retsd = rets.loc[start:end].copy()
retsd.dropna(inplace=True)

In [ ]:
retsf = np.log(factors / factors.shift(1))
retsf = retsf.loc[start:end]
retsf.dropna(inplace=True)
retsf = retsf.loc[retsd.index].dropna()

In [ ]:
retsf.corr()

In [ ]:
res = pd.DataFrame()

In [ ]:
np.set_printoptions(formatter={'float': lambda x: f'{x:5.2f}'})

In [ ]:
split = int(len(retsf) * 0.5)
for sym in rets.columns[:4]:
    print('\n' + sym)
    print(74 * '=')
    retsf_, retsd_ = retsf.iloc[:split], retsd.iloc[:split]
    reg = np.linalg.lstsq(retsf_, retsd_[sym], rcond=-1)[0]   
    retsf_, retsd_ = retsf.iloc[split:], retsd.iloc[split:]
    mu_apt = np.dot(retsf_.mean() * 252, reg)
    mu_real =  retsd_[sym].mean() * 252
    res = pd.concat([res, pd.DataFrame({'mu_apt': mu_apt,
                    'mu_real': mu_real}, index=[sym,])],
                    sort=True)
    print('fl: {} | apt: {:.3f} | real: {:.3f}'
          .format(reg.round(1), mu_apt, mu_real))

In [ ]:
res.plot(kind='bar', figsize=(10, 6));

In [ ]:
sym

In [ ]:
rets_sym = np.dot(retsf_, reg)

In [ ]:
rets_sym = pd.DataFrame(rets_sym,
                        columns=[sym + '_apt'],
                        index=retsf_.index)

In [ ]:
rets_sym[sym + '_real'] = retsd_[sym]

In [ ]:
rets_sym.mean() * 252

In [ ]:
rets_sym.std() * 252 ** 0.5

In [ ]:
rets_sym.corr()

In [ ]:
rets_sym.cumsum().apply(np.exp).plot(figsize=(10, 6));

In [ ]:
rets_sym['same'] = (np.sign(rets_sym[sym + '_apt']) ==
                    np.sign(rets_sym[sym + '_real']))

In [ ]:
rets_sym['same'].value_counts()

In [ ]:
rets_sym['same'].value_counts()[True] / len(rets_sym)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>